In [41]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph
from openai import AzureOpenAI

Conectando ao Neo4j e aos Azure OpenAI

In [42]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [43]:
# Conectando o modelo de Embedding

#client_small = AzureOpenAI(
#  api_key = os.getenv("OPENAI_API_KEY"),
#  azure_endpoint = os.getenv("EMBEDDING_SMALL_ENDPOINT"),
#  api_version = os.getenv("API_VERSION"))

client_large = AzureOpenAI(
  api_key = os.getenv("OPENAI_API_KEY"),
  azure_endpoint = os.getenv("EMBEDDING_LARGE_ENDPOINT"),
  api_version = os.getenv("API_VERSION"))


# The function receives a sencence and returns the embedding (1D numpy array)
def get_embeddings_openAI(text):
    #model = "text-embedding-3-small"
    #embedding_small = client_small.embeddings.create(input = text, model=model)
    #return embedding_small.data
    model = "text-embedding-3-large"
    embedding_large = client_large.embeddings.create(input = text, model=model)
    return embedding_large.data 

### GRAPH RAG  
  
Definir uma funcao que: 1) recebe um texto; 2)usa o modelo de embedding para transformar o texto em um vetor: 3) Usa o vetor para buscar outros textos do grafo (usando o índice de vetores)   

In [44]:
def busca_grafo_vetor(query_text):
    query_embedding = get_embeddings_openAI(query_text)[0].embedding

    # Buscando no índice de vetores 
    results = graph.query("""
        CALL db.index.vector.queryNodes('Thesis_Embeddings', 15, """ + str(query_embedding) + """)
        YIELD node, score
        MATCH (node)-[:author]-(author)
        RETURN node.uri, node.title, node.abstract, node.created, author.label, score
        """)
    
    return (results)

In [45]:
def busca_grafo_fulltext(query_text):

    results = graph.query("""
        CALL db.index.fulltext.queryNodes("Thesis_fulltext", '""" + query_text + """') 
        YIELD node, score
        MATCH (node)-[:author]-(author)
        RETURN node.uri, node.title, node.abstract, node.created , author.label, score 
        LIMIT 15 
        """)
    return (results)

In [46]:
# Reciprocal Rank Fusion (RRF)
def RFF(rank1, rank2, w_rank1=1.0, w_rank2=1.0):
    k = 60
    score = {}
    title_text = {}

    for p in range(len(rank1)):
        uri = rank1[p]['node.uri']
        score[uri] = 1/(p+1+k)
        title_text[uri] = {'title': rank1[p]['node.title'], 
                           'text': rank1[p]['node.abstract'], 
                           'author': rank1[p]['author.label'],
                           'created': rank1[p]['node.created']}

    for p in range(len(rank2)):
        uri = rank2[p]['node.uri']
        if uri not in score:
            score[uri] = 1/(p+1+k)
            title_text[uri] = {'title': rank2[p]['node.title'], 
                               'text': rank2[p]['node.abstract'], 
                               'author': rank2[p]['author.label'],
                               'created': rank2[p]['node.created']}
        else:
            score[uri] = (score[uri]) * w_rank1 + (1/(p+1+k)) * w_rank2


    uri_list = []
    score_list = []
    sorted_title_text = []
    for i in sorted(score, key = score.get, reverse=True):
        uri_list.append(i)
        score_list.append(score[i])
        sorted_title_text.append(title_text[i])
    return (sorted_title_text, score_list)



In [47]:
def busca_grafo_hibrida(query_text):
    results_vector = busca_grafo_vetor(query_text)
    results_text = busca_grafo_fulltext(query_text)
    return RFF(results_vector, results_text, w_rank1=1.0, w_rank2=0.5)

In [48]:
def gen_respostas(query_text):

    results = busca_grafo_hibrida(query_text)

    #endpoint = os.getenv("GPT35_ENDPOINT")  
    #deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo")
    endpoint = os.getenv("GPT4_ENDPOINT")  
    deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4") 
    subscription_key = os.getenv("OPENAI_API_KEY")   
        
    
            
    client_chat = AzureOpenAI(  
            azure_endpoint=endpoint,  
            api_key=subscription_key,  
            api_version="2024-05-01-preview",  
        )  

    respostas = []

    for n in range(5):
        text = results[0][n]['text']
        titulo = results[0][n]['title']
        autor = results[0][n]['author']
        created = results[0][n]['created']
        citation = autor + ". " + titulo + ". " + created + "."

        completion = client_chat.chat.completions.create(  
        model=deployment,  
        messages=[
            {
            "role": "system",
            "content": "You will be provided with a text delimited by triple quotes and a question. Your task is to answer the question using only the provided text and to cite the passage(s) of the document used to answer the question. If the text does not contain the information needed to answer this question then simply write: 'Informação Insuficiente.' If an answer to the question is provided, it must be annotated with a citation. Use the following format for to cite relevant passages (" + citation + "). Your answers must be written in Brazilian Portuguese."
            #"content": "Use the provided text delimited by triple quotes to answer questions. If the answer cannot be found in the articles, write {NONE}"
            #"content": "INSTRUCTIONS:\nAnswer the users QUESTION using the CONTEXT text below.\nKeep your answer ground in the facts of the CONTEX.\n Very important: If the CONTEXT doesn’t contain the facts to answer the QUESTION return {NONE}.\n\nCONTEXT:\n" + text
            },
            {
            "role": "user",
            "content": '"""' + text + '""" "QUESTION: "' + query_text
            }
            ],  
                #past_messages=10,  
                max_tokens=800,  
                temperature=1.0,  
                top_p=0.95,  
                frequency_penalty=0,  
                presence_penalty=0,  
                stop=None,  
                stream=False  
            )  

        #resposta = completion.to_json()
        resposta = completion.to_dict()['choices'][0]['message']['content']
        respostas.append(resposta)
        
    return respostas

In [72]:
def summarize_respostas(query_text, respostas):

    fontes = ''
    for r in respostas:
        fontes = fontes + '\n' + r

    #endpoint = os.getenv("GPT35_ENDPOINT")  
    #deployment = os.getenv("DEPLOYMENT_NAME", "gpt-35-turbo")
    endpoint = os.getenv("GPT4_ENDPOINT")  
    deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4") 
    subscription_key = os.getenv("OPENAI_API_KEY") 
        
    
            
    client_chat = AzureOpenAI(  
            azure_endpoint=endpoint,  
            api_key=subscription_key,  
            api_version="2024-05-01-preview",  
        )  

    completion = client_chat.chat.completions.create(  
    model=deployment,  
    messages=[
        {
        "role": "system",
        "content": "You will be provided with one question and several answers for the question delimited by triple quotes. Your task is to summarize the provided answers. You must maintain the citations and references for the original documents using the same format. Include facts present in the provided answers that directly address the provided question in the final answer. Your answers must be written in Brazilian Portuguese."
        #"content": "INSTRUCTIONS:\n Sumarize os textos do contexto, referenciando as fontes ao final em uma bibliografia. Não mencione nem use como fonte os textos que não falem sobre " + personagem + "."
        },
        {
        "role": "user",
        "content": '"""' + fontes + '"""' + 'QUESTION: ' + query_text
        }
        ],  
            #past_messages=10,  
            max_tokens=800,  
            temperature=1.0,  
            top_p=0.95,  
            frequency_penalty=0,  
            presence_penalty=0,  
            stop=None,  
            stream=False  
        )  

    #resposta = completion.to_json()
    summ_resposta = completion.to_dict()['choices'][0]['message']['content']
        
    return summ_resposta

In [73]:
# Lista de perguntas

def perguntas(personagem):
    perg = []
    perg.append('Qual o nome completo de ' + personagem + '?')
    perg.append('Onde nasceu ' + personagem + '?')
    perg.append('Qual a data de nascimento de ' + personagem + '?')
    perg.append('Onde morreu ' + personagem + '?')
    perg.append('Qual a data de morte de ' + personagem + '?')
    perg.append('Quem foi ' + personagem + '?')
    perg.append('Quais os principais feitos de ' + personagem + '?')
    perg.append('Quais os principais cargos, funções, ou emprogos de ' + personagem + '?')
    
    return perg

In [74]:
def perguntas_e_respostas(personagem):

    perg_resp = {}

    pergs = perguntas(personagem)

    for perg in pergs:
        respostas = gen_respostas(perg)
        resposta_final = summarize_respostas(perg, respostas)
        perg_resp[perg] = {"Resumo": resposta_final, "Respostas": respostas} 

    return perg_resp

In [ ]:
personagem = 'Padre Vieira'
dic_perguntas_respostas = perguntas_e_respostas(personagem)

In [58]:
print(dic_perguntas_respostas.keys())

dict_keys(['Qual o nome completo de Padre Vieira?', 'Onde nasceu Padre Vieira?', 'Qual a data de nascimento de Padre Vieira?', 'Onde morreu Padre Vieira?', 'Qual a data de morte de Padre Vieira?', 'Quem foi Padre Vieira?', 'Quais os principais feitos de Padre Vieira?', 'Quais os principais cargos, funções, ou emprogos de Padre Vieira?'])


In [65]:
dic_perguntas_respostas['Quais os principais cargos, funções, ou emprogos de Padre Vieira?']

{'Resumo': 'A resposta fornecida não contém informações suficientes para determinar os principais cargos, funções ou empregos de Padre Vieira.',
 'Respostas': ['Informação Insuficiente.',
  'Informação Insuficiente.',
  'Informação Insuficiente.',
  'Informação Insuficiente.',
  'Informação Insuficiente.']}

In [78]:
perg = "Quem foram os irmão Breves?"
respostas = gen_respostas(perg)
resposta_final = summarize_respostas(perg, respostas)

In [79]:
resposta_final

'Os irmãos Breves, José e Joaquim de Souza Breves, foram figuras importantes no cenário escravista brasileiro do século XIX. Eles são conhecidos como Comendadores e suas trajetórias são essenciais para compreender aspectos da sociedade escravista daquele período, incluindo a acumulação de suas riquezas, suas atuações na política imperial e seu envolvimento no tráfico ilegal de escravos africanos (Lourenço, Thiago Campos Pessoa. O império dos Souza Breves nos oitocentos: política e escravidão nas trajetórias dos Comendadores José e Joaquim de Souza Breves. 2010.).'

In [80]:
respostas

['Os irmãos Breves foram José e Joaquim de Souza Breves. Eles são destacados no texto como personagens centrais no universo escravista brasileiro do século XIX e são conhecidos como Comendadores. As trajetórias deles são utilizadas para entender questões relacionadas à sociedade escravista da época, incluindo a construção de suas fortunas, suas inserções na política imperial, e sua participação no tráfico ilegal de africanos (Lourenço, Thiago Campos Pessoa. O império dos Souza Breves nos oitocentos: política e escravidão nas trajetórias dos Comendadores José e Joaquim de Souza Breves. 2010.).',
 'Informação Insuficiente.',
 'Informação Insuficiente.',
 'Informação Insuficiente.',
 'Informação Insuficiente.']